In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import mysql.connector


In [2]:
mydb = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    passwd = 'password',
    database = 'hotels'
)

In [3]:
mycursor = mydb.cursor()

# create function
def sql(yourQuery):
    mycursor.execute(yourQuery)
    myresult = mycursor.fetchall()
    df = pd.DataFrame(myresult, columns = mycursor.column_names)
    return df

In [4]:
sql(
    '''
    SELECT * FROM dataset;
''')

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,room_type_reserved,lead_time,arrival_date,market_segment_type,avg_price_per_room,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,Room_Type 1,224,2017-10-02,Offline,65.00,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,Room_Type 1,5,2018-11-06,Online,106.68,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,Room_Type 1,1,2018-02-28,Online,60.00,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,Room_Type 1,211,2018-05-20,Online,100.00,Canceled
4,INN00005,2,0,1,1,Not Selected,Room_Type 1,48,2018-04-11,Online,94.50,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...
695,INN00696,2,0,0,2,Meal Plan 2,Room_Type 1,257,2017-07-01,Online,101.50,Canceled
696,INN00697,2,0,2,4,Not Selected,Room_Type 1,207,2018-07-28,Online,80.75,Not_Canceled
697,INN00698,2,0,2,2,Meal Plan 1,Room_Type 1,108,2018-12-25,Online,104.00,Not_Canceled
698,INN00699,2,0,0,3,Not Selected,Room_Type 1,39,2018-03-15,Online,51.19,Not_Canceled


### 1. What is the total number of reservations in the dataset?


In [5]:
sql('''
    SELECT COUNT(Booking_ID) AS Total_Reservations
FROM dataset;
''')

,Total_Reservations
0,700


### 2. Which meal plan is the most popular among guests?


In [6]:
sql('''
SELECT type_of_meal_plan, COUNT(*) AS Total_Reservations
FROM dataset
GROUP BY type_of_meal_plan
ORDER BY Total_Reservations DESC
LIMIT 1;
''')

,type_of_meal_plan,Total_Reservations
0,Meal Plan 1,527


### 3. What is the average price per room for reservations involving children?

In [7]:
sql('''
SELECT AVG(avg_price_per_room) AS Avg_Price_Per_Room_With_Children
FROM dataset
WHERE no_of_children > 0;
''')

,Avg_Price_Per_Room_With_Children
0,144.568333


### 4. How many reservations were made for the year 2017 

In [8]:
sql('''
SELECT COUNT(Booking_ID) AS Reservations_2017
FROM dataset
WHERE YEAR(arrival_date) = 2017;
''')

,Reservations_2017
0,123


### 5. What is the most commonly booked room type?

In [9]:
sql('''
SELECT room_type_reserved, COUNT(*) AS Total_Bookings
FROM dataset
GROUP BY room_type_reserved
ORDER BY Total_Bookings DESC
LIMIT 1;

''')

,room_type_reserved,Total_Bookings
0,Room_Type 1,534


###  6. How many reservations fall on a weekend (no_of_weekend_nights > 0)?

In [10]:
sql('''
SELECT COUNT(*) AS Weekend_Reservations
FROM dataset
WHERE no_of_weekend_nights > 0;
''')

,Weekend_Reservations
0,383


### 7. What is the highest and lowest lead time for reservations?

In [11]:
sql('''
SELECT 
    MAX(lead_time) AS Highest_Lead_Time,
    MIN(lead_time) AS Lowest_Lead_Time
FROM dataset;
''')

,Highest_Lead_Time,Lowest_Lead_Time
0,443,0


### 8. What is the most common market segment type for reservations?

In [12]:
sql('''
SELECT market_segment_type, COUNT(*) AS Total_Reservations
FROM dataset
GROUP BY market_segment_type
ORDER BY Total_Reservations DESC
LIMIT 1;
''')

,market_segment_type,Total_Reservations
0,Online,518


###  9. How many reservations have a booking status of "Confirmed"?

In [13]:
sql('''
SELECT COUNT(*) AS Confirmed_Reservations
FROM dataset
WHERE booking_status = 'Confirmed';
''')

,Confirmed_Reservations
0,0


### 10. What is the total number of adults and children across all reservations?


In [15]:
sql('''
SELECT 
    SUM(no_of_adults) AS Total_Adults,
    SUM(no_of_children) AS Total_Children,
    SUM(no_of_adults + no_of_children) AS Total_Adults_And_Children
FROM dataset;
''')

,Total_Adults,Total_Children,Total_Adults_And_Children
0,1316,69,1385


### 11. What is the average number of weekend nights for reservations involving children?

In [16]:
sql('''
SELECT AVG(no_of_weekend_nights) AS Avg_Weekend_Nights_With_Children
FROM dataset
WHERE no_of_children > 0;
''')

,Avg_Weekend_Nights_With_Children
0,1.0000


### 12. How many reservations were made in each month of the year?

In [17]:
sql('''
SELECT 
    EXTRACT(MONTH FROM arrival_date) AS Month,
    COUNT(*) AS Reservations
FROM dataset
GROUP BY EXTRACT(MONTH FROM arrival_date)
ORDER BY Month;
''')

,Month,Reservations
0,1,11
1,2,28
2,3,52
3,4,67
4,5,55
5,6,84
6,7,44
7,8,70
8,9,80
9,10,103


### 13. What is the average number of nights (both weekend and weekday) spent by guests for each room type?

In [18]:
sql('''
SELECT 
    room_type_reserved,
    AVG(no_of_weekend_nights + no_of_week_nights) AS Avg_Nights
FROM dataset
GROUP BY room_type_reserved;
''')

,room_type_reserved,Avg_Nights
0,Room_Type 1,2.8783
1,Room_Type 4,3.8000
2,Room_Type 2,3.0000
3,Room_Type 6,3.6111
4,Room_Type 5,2.5000
5,Room_Type 7,2.6667


### 14. For reservations involving children, what is the most common room type, and what is the average price for that room type?

In [19]:
sql('''
WITH Children_Reservations AS (
    SELECT *
    FROM dataset
    WHERE no_of_children > 0
)

SELECT 
    room_type_reserved,
    COUNT(*) AS Total_Reservations,
    AVG(avg_price_per_room) AS Average_Price
FROM Children_Reservations
GROUP BY room_type_reserved
ORDER BY Total_Reservations DESC
LIMIT 1;
''')

,room_type_reserved,Total_Reservations,Average_Price
0,Room_Type 1,24,123.122917


### 15. Find the market segment type that generates the highest average price per room.

In [20]:
sql('''
SELECT 
    market_segment_type,
    AVG(avg_price_per_room) AS Avg_Price_Per_Room
FROM dataset
GROUP BY market_segment_type
ORDER BY Avg_Price_Per_Room DESC
LIMIT 1;
''')

,market_segment_type,Avg_Price_Per_Room
0,Online,112.455212
